In [68]:
from ctypes import *
import random
from PIL import Image, ImageOps
import numpy as np
import os
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, accuracy_score

my_dll = cdll.LoadLibrary(
        "C:/Users/ariel/OneDrive/Bureau/ESGI/Bachelor/Projet Annuel/Library C++/Library_MLP/cmake-build-debug/Library_MLP.dll")

In [69]:
def create_mlp_model(struct_model):
    struct_model_int = [int(i) for i in struct_model]
    struct_model_type = len(struct_model_int) * c_int

    my_dll.create_mlp_model.argtypes = [struct_model_type,
                                        c_int]
    my_dll.create_mlp_model.restype = POINTER(c_void_p)

    return my_dll.create_mlp_model(struct_model_type(*struct_model_int), len(struct_model_int))

In [70]:
def predict_mlp(model, input, len_input, is_classification):
    my_dll.predict_mlp_model.argtypes = [c_void_p, POINTER(c_float), c_int32,
                                         c_int32]
    my_dll.predict_mlp_model.restype = POINTER(c_float)
    ng = c_float * len(input)
    predictions = my_dll.predict_mlp_model(model, ng(*input), len_input, is_classification)
    return predictions[1]

In [71]:
def train_mlp(model_to_train, X_to_train, y, learning_rate, nb_iter, is_classification):
    my_dll.alternate_train_MLP.argtypes = [c_void_p, POINTER(c_float),
                                           POINTER(c_float), c_float, c_int32]
    my_dll.alternate_train_MLP.restype = None

    for i in tqdm(range(0, nb_iter)):
        k = random.choice(list(range(0, len(X_to_train))))
        arr_types_X = c_float * len(X_to_train[k])
        arr_types_Y = c_float * len(y[k])
        my_dll.alternate_train_MLP(model_to_train, arr_types_X(*X_to_train[k]), arr_types_Y(*y[k]), learning_rate,
                                   is_classification)

In [72]:
def destroy_model(model_to_destroy):
    my_dll.destroy_mlp_model.argtypes = [c_void_p]
    my_dll.destroy_mlp_model.restype = None

    my_dll.destroy_mlp_model(model_to_destroy)

In [73]:
# Linear Simple
X = np.array([
      [1, 1],
      [2, 3],
      [3, 3]
])
Y = np.array([
      1,
      -1,
      -1
])
Y = [[i] for i in Y]

npl = [2, 2, 1]

model = create_mlp_model(npl)

print("Before Training :")
for k, i in enumerate(X):
    prediction = predict_mlp(model, i, len(i), 1)
    print(f"True : {Y[k]} Pred : {prediction}")

print("\nTraining...")
train_mlp(model, X, Y, 0.1, 10000, 1)

print("After Training :")
for k, i in enumerate(X):
    prediction = predict_mlp(model, i, len(i), 1)
    print(f"True : {Y[k]} Pred : {prediction}")

destroy_model(model)

 17%|████████████▍                                                             | 1685/10000 [00:00<00:00, 16779.92it/s]

Before Training :
True : [1] Pred : 0.6578037738800049
True : [-1] Pred : 0.6627044677734375
True : [-1] Pred : 0.6894180774688721

Training...


100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 14643.77it/s]

After Training :
True : [1] Pred : 0.9850987195968628
True : [-1] Pred : -0.9885993599891663
True : [-1] Pred : -0.9898900985717773


In [74]:
# Linear Multiple
X = np.concatenate([np.random.random((50,2)) * 0.9 + np.array([1, 1]), np.random.random((50,2)) * 0.9 + np.array([2, 2])])
Y = np.concatenate([np.ones((50, 1)), np.ones((50, 1)) * -1.0])

Y = [[i] for i in Y]

npl = [2, 2, 1]

model = create_mlp_model(npl)

print("Before Training :")
for k, i in enumerate(X):
    prediction = predict_mlp(model, i, len(i), 1)
    print(f"True : {Y[k]} Pred : {prediction}")

print("\nTraining...")
train_mlp(model, X, Y, 0.01, 10000, 1)

print("After Training :")
for k, i in enumerate(X):
    prediction = predict_mlp(model, i, len(i), 1)
    print(f"True : {Y[k]} Pred : {prediction}")

destroy_model(model)

Before Training :
True : [array([1.])] Pred : 0.6751554608345032
True : [array([1.])] Pred : 0.6578338742256165
True : [array([1.])] Pred : 0.6812557578086853
True : [array([1.])] Pred : 0.6722918748855591
True : [array([1.])] Pred : 0.667424738407135
True : [array([1.])] Pred : 0.681095540523529
True : [array([1.])] Pred : 0.6801173686981201
True : [array([1.])] Pred : 0.6708577871322632
True : [array([1.])] Pred : 0.6709064245223999
True : [array([1.])] Pred : 0.6782581210136414
True : [array([1.])] Pred : 0.6820796728134155
True : [array([1.])] Pred : 0.6584316492080688
True : [array([1.])] Pred : 0.6732794046401978
True : [array([1.])] Pred : 0.669552206993103
True : [array([1.])] Pred : 0.6545571088790894
True : [array([1.])] Pred : 0.6778918504714966
True : [array([1.])] Pred : 0.6682384014129639
True : [array([1.])] Pred : 0.6849907040596008
True : [array([1.])] Pred : 0.6615824699401855
True : [array([1.])] Pred : 0.6659664511680603
True : [array([1.])] Pred : 0.686247229576110

  0%|                                                                                        | 0/10000 [00:00<?, ?it/s]

True : [array([-1.])] Pred : 0.681662917137146
True : [array([-1.])] Pred : 0.6900631785392761
True : [array([-1.])] Pred : 0.685897946357727
True : [array([-1.])] Pred : 0.6892054080963135

Training...


100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 18885.20it/s]


After Training :
True : [array([1.])] Pred : 0.9497362375259399
True : [array([1.])] Pred : 0.9919587969779968
True : [array([1.])] Pred : 0.9632875919342041
True : [array([1.])] Pred : 0.99267578125
True : [array([1.])] Pred : 0.9314562082290649
True : [array([1.])] Pred : 0.8276677131652832
True : [array([1.])] Pred : 0.988238513469696
True : [array([1.])] Pred : 0.9825795888900757
True : [array([1.])] Pred : 0.994976282119751
True : [array([1.])] Pred : 0.995514988899231
True : [array([1.])] Pred : 0.9941143989562988
True : [array([1.])] Pred : 0.996531069278717
True : [array([1.])] Pred : 0.9506921768188477
True : [array([1.])] Pred : 0.8887604475021362
True : [array([1.])] Pred : 0.9929636120796204
True : [array([1.])] Pred : 0.7369463443756104
True : [array([1.])] Pred : 0.9303804039955139
True : [array([1.])] Pred : 0.9561740756034851
True : [array([1.])] Pred : 0.9606090188026428
True : [array([1.])] Pred : 0.9937060475349426
True : [array([1.])] Pred : 0.9139853119850159
True 

In [75]:
# XOR
X = np.array([[1, 0], [0, 1], [0, 0], [1, 1]])
Y = np.array([1, 1, -1, -1])
Y = [[i] for i in Y]

npl = [2, 2, 1]

model = create_mlp_model(npl)

print("Before Training :")
for k, i in enumerate(X):
    prediction = predict_mlp(model, i, len(i), 1)
    print(f"True : {Y[k]} Pred : {prediction}")

print("\nTraining...")
train_mlp(model, X, Y, 0.1, 10000, 1)

print("After Training :")
for k, i in enumerate(X):
    prediction = predict_mlp(model, i, len(i), 1)
    print(f"True : {Y[k]} Pred : {prediction}")

destroy_model(model)

Before Training :
True : [1] Pred : 0.6695356965065002


 20%|███████████████                                                           | 2035/10000 [00:00<00:00, 20155.97it/s]

True : [1] Pred : 0.5567963123321533
True : [-1] Pred : 0.5449346303939819
True : [-1] Pred : 0.6578037738800049

Training...


100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 16202.53it/s]

After Training :
True : [1] Pred : -0.07907441258430481
True : [1] Pred : 0.9668468832969666
True : [-1] Pred : -0.9734566807746887
True : [-1] Pred : -0.07616569846868515


In [76]:
# Cross
X = np.random.random((500, 2)) * 2.0 - 1.0
Y = np.array([1 if abs(p[0]) <= 0.3 or abs(p[1]) <= 0.3 else -1 for p in X])
Y = [[i] for i in Y]

npl = [2, 4, 1]

model = create_mlp_model(npl)

print("Before Training :")
for k, i in enumerate(X):
    prediction = predict_mlp(model, i, len(i), 1)
    print(f"True : {Y[k]} Pred : {prediction}")

print("\nTraining...")
train_mlp(model, X, Y, 0.1, 100000, 1)

print("After Training :")
for k, i in enumerate(X):
    prediction = predict_mlp(model, i, len(i), 1)
    print(f"True : {Y[k]} Pred : {prediction}")

destroy_model(model)

Before Training :
True : [-1] Pred : 0.22701764106750488
True : [-1] Pred : 0.3618398606777191
True : [1] Pred : 0.29239633679389954
True : [1] Pred : 0.37658441066741943
True : [-1] Pred : 0.4315589666366577
True : [-1] Pred : 0.3284955620765686
True : [-1] Pred : 0.2322077453136444
True : [1] Pred : 0.3196714520454407
True : [1] Pred : 0.30262619256973267
True : [1] Pred : 0.3894740045070648
True : [1] Pred : 0.29786813259124756
True : [1] Pred : 0.2898252606391907
True : [1] Pred : 0.2910308539867401
True : [-1] Pred : 0.35097536444664
True : [-1] Pred : 0.23477742075920105
True : [1] Pred : 0.272663950920105
True : [1] Pred : 0.2632316052913666
True : [-1] Pred : 0.41841229796409607
True : [1] Pred : 0.3410935699939728
True : [1] Pred : 0.31650009751319885
True : [1] Pred : 0.4137206971645355
True : [-1] Pred : 0.32294580340385437
True : [1] Pred : 0.2820994555950165
True : [1] Pred : 0.3693934977054596
True : [1] Pred : 0.2785819172859192
True : [1] Pred : 0.30991002917289734
True

True : [-1] Pred : 0.2566937208175659
True : [-1] Pred : 0.3300985097885132
True : [-1] Pred : 0.4365857243537903
True : [-1] Pred : 0.2674519121646881
True : [-1] Pred : 0.4312562644481659
True : [-1] Pred : 0.259372740983963
True : [-1] Pred : 0.289140522480011
True : [1] Pred : 0.2510024905204773
True : [-1] Pred : 0.2320331335067749
True : [-1] Pred : 0.24537062644958496
True : [-1] Pred : 0.35615310072898865
True : [-1] Pred : 0.3523922860622406
True : [1] Pred : 0.3571781814098358
True : [1] Pred : 0.25344330072402954
True : [1] Pred : 0.29153692722320557
True : [1] Pred : 0.2835223078727722
True : [1] Pred : 0.3548855781555176
True : [1] Pred : 0.29859909415245056
True : [1] Pred : 0.32645153999328613
True : [1] Pred : 0.36415982246398926
True : [-1] Pred : 0.360571950674057
True : [1] Pred : 0.2907356321811676
True : [-1] Pred : 0.3269251585006714
True : [1] Pred : 0.3308917284011841
True : [1] Pred : 0.2729795575141907
True : [-1] Pred : 0.3512987494468689
True : [1] Pred : 0.

100%|███████████████████████████████████████████████████████████████████████| 100000/100000 [00:06<00:00, 15460.04it/s]


After Training :
True : [-1] Pred : 0.4407234787940979
True : [-1] Pred : -0.7841074466705322
True : [1] Pred : 0.9999986290931702
True : [1] Pred : 0.9999958872795105
True : [-1] Pred : 0.09729597717523575
True : [-1] Pred : -1.0
True : [-1] Pred : -0.02760852873325348
True : [1] Pred : 1.0
True : [1] Pred : 1.0
True : [1] Pred : 0.9999833106994629
True : [1] Pred : 1.0
True : [1] Pred : 1.0
True : [1] Pred : 1.0
True : [-1] Pred : -1.0
True : [-1] Pred : -0.33506089448928833
True : [1] Pred : 1.0
True : [1] Pred : 1.0
True : [-1] Pred : 0.14904598891735077
True : [1] Pred : 1.0
True : [1] Pred : 0.1116701066493988
True : [1] Pred : 0.9962221384048462
True : [-1] Pred : -0.6193961501121521
True : [1] Pred : 1.0
True : [1] Pred : 1.0
True : [1] Pred : 1.0
True : [1] Pred : 0.48177582025527954
True : [1] Pred : -0.1625625491142273
True : [-1] Pred : -1.0
True : [-1] Pred : 0.17296336591243744
True : [1] Pred : 1.0
True : [1] Pred : 1.0
True : [1] Pred : -0.3160497546195984
True : [-1] P

In [77]:
# Multi Linear 3 classes
X = np.random.random((500, 2)) * 2.0 - 1.0
Y = np.array([[1, 0, 0] if -p[0] - p[1] - 0.5 > 0 and p[1] < 0 and p[0] - p[1] - 0.5 < 0 else 
              [0, 1, 0] if -p[0] - p[1] - 0.5 < 0 and p[1] > 0 and p[0] - p[1] - 0.5 < 0 else 
              [0, 0, 1] if -p[0] - p[1] - 0.5 < 0 and p[1] < 0 and p[0] - p[1] - 0.5 > 0 else 
              [0, 0, 0]for p in X])

X = X[[not np.all(arr == [0, 0, 0]) for arr in Y]]
Y = Y[[not np.all(arr == [0, 0, 0]) for arr in Y]]

Y = [[i] for i in Y]

npl = [2, 3, 1]

model = create_mlp_model(npl)

print("Before Training :")
for k, i in enumerate(X):
    prediction = predict_mlp(model, i, len(i), 1)
    print(f"True : {Y[k]} Pred : {prediction}")

print("\nTraining...")
train_mlp(model, X, Y, 0.1, 100000, 1)

print("After Training :")
for k, i in enumerate(X):
    prediction = predict_mlp(model, i, len(i), 1)
    print(f"True : {Y[k]} Pred : {prediction}")

destroy_model(model)

Before Training :
True : [array([0, 1, 0])] Pred : -4.2201683186052844e+37
True : [array([1, 0, 0])] Pred : -4.2201683186052844e+37
True : [array([0, 1, 0])] Pred : -4.2201683186052844e+37
True : [array([1, 0, 0])] Pred : -4.2201683186052844e+37
True : [array([0, 0, 1])] Pred : -4.2201683186052844e+37
True : [array([0, 1, 0])] Pred : -4.2201683186052844e+37
True : [array([0, 1, 0])] Pred : -4.2201683186052844e+37
True : [array([0, 1, 0])] Pred : -4.2201683186052844e+37
True : [array([0, 1, 0])] Pred : -4.2201683186052844e+37
True : [array([1, 0, 0])] Pred : -4.2201683186052844e+37
True : [array([0, 0, 1])] Pred : -4.2201683186052844e+37
True : [array([1, 0, 0])] Pred : -4.2201683186052844e+37
True : [array([0, 1, 0])] Pred : -4.2201683186052844e+37
True : [array([1, 0, 0])] Pred : -4.2201683186052844e+37
True : [array([0, 1, 0])] Pred : -4.2201683186052844e+37
True : [array([0, 1, 0])] Pred : -4.2201683186052844e+37
True : [array([1, 0, 0])] Pred : -4.2201683186052844e+37
True : [array

True : [array([0, 1, 0])] Pred : -4.2201683186052844e+37
True : [array([0, 1, 0])] Pred : -4.2201683186052844e+37
True : [array([0, 1, 0])] Pred : -4.2201683186052844e+37
True : [array([0, 1, 0])] Pred : -4.2201683186052844e+37
True : [array([0, 1, 0])] Pred : -4.2201683186052844e+37
True : [array([0, 1, 0])] Pred : -4.2201683186052844e+37
True : [array([0, 1, 0])] Pred : -4.2201683186052844e+37
True : [array([1, 0, 0])] Pred : -4.2201683186052844e+37
True : [array([1, 0, 0])] Pred : -4.2201683186052844e+37
True : [array([0, 1, 0])] Pred : -4.2201683186052844e+37
True : [array([1, 0, 0])] Pred : -4.2201683186052844e+37
True : [array([0, 0, 1])] Pred : -4.2201683186052844e+37


  0%|                                                                                       | 0/100000 [00:00<?, ?it/s]

True : [array([1, 0, 0])] Pred : -4.2201683186052844e+37
True : [array([0, 0, 1])] Pred : -4.2201683186052844e+37
True : [array([0, 1, 0])] Pred : -4.2201683186052844e+37
True : [array([0, 1, 0])] Pred : -4.2201683186052844e+37
True : [array([0, 1, 0])] Pred : -4.2201683186052844e+37
True : [array([1, 0, 0])] Pred : -4.2201683186052844e+37
True : [array([0, 1, 0])] Pred : -4.2201683186052844e+37
True : [array([1, 0, 0])] Pred : -4.2201683186052844e+37
True : [array([0, 0, 1])] Pred : -4.2201683186052844e+37
True : [array([0, 0, 1])] Pred : -4.2201683186052844e+37
True : [array([1, 0, 0])] Pred : -4.2201683186052844e+37
True : [array([0, 1, 0])] Pred : -4.2201683186052844e+37
True : [array([0, 1, 0])] Pred : -4.2201683186052844e+37
True : [array([0, 1, 0])] Pred : -4.2201683186052844e+37
True : [array([0, 0, 1])] Pred : -4.2201683186052844e+37
True : [array([0, 1, 0])] Pred : -4.2201683186052844e+37
True : [array([0, 0, 1])] Pred : -4.2201683186052844e+37
True : [array([0, 0, 1])] Pred 

TypeError: only size-1 arrays can be converted to Python scalars

In [86]:
X = np.array([
      [1],
      [2]
])
Y = np.array([
      2,
      3
])
Y = [[i] for i in Y]

npl = [2, 1, 1]

model = create_mlp_model(npl)

print("Before Training :")
for k, i in enumerate(X):
    prediction = predict_mlp(model, i, len(i), 0)
    print(f"True : {Y[k]} Pred : {prediction}")

print("\nTraining...")
train_mlp(model, X, Y, 0.001, 1000, 0)

print("After Training :")
for k, i in enumerate(X):
    prediction = predict_mlp(model, i, len(i), 0)
    print(f"True : {Y[k]} Pred : {prediction}")

destroy_model(model)


100%|███████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 28316.74it/s]

Before Training :
True : [2] Pred : 0.12234988063573837
True : [3] Pred : 0.1200086772441864

Training...
After Training :
True : [2] Pred : 1.69770348072052
True : [3] Pred : 1.7620220184326172
